In [ ]:
import sys
from pathlib import Path

path_root = Path( '/project_ghent/HADSP/reservoirpy/')
sys.path.append(str(path_root))
path_root = Path( '/project_ghent/HADSP/hadsp/')
sys.path.append(str(path_root))

In [ ]:
import numpy as np
from scipy import sparse
from tqdm import tqdm

from matplotlib import pyplot as plt
from seaborn import heatmap, color_palette

In [ ]:
from importlib import reload

# Utility functions
from reservoir.utility import show_matrice

# SEED
SEED = 49387

# Reservoir functions

In [ ]:
from reservoir.reservoir import update_reservoir, ridge_regression, train, run
from reservoir.activation_functions import tanh, heaviside, sigmoid

# the activation function choosen for the rest of the experiment
# activation_function = lambda x : sigmoid(2*(x-0.5))
activation_function = lambda x : tanh(x)

plt.plot(np.linspace(0, 2, 100), activation_function(np.linspace(0, 2, 100)))
plt.grid()

# Datasets

Lots of different on availabale : https://towardsdatascience.com/a-data-lakes-worth-of-audio-datasets-b45b88cd4ad

Classification: 
https://arxiv.org/abs/1803.07870

https://github.com/FilippoMB/Time-series-classification-and-clustering-with-Reservoir-Computing

Multivariate:
https://www.timeseriesclassification.com/dataset.php

## Mackey-Glass

In [ ]:
from reservoirpy.datasets import mackey_glass

timesteps = 10000
mg_inputs = mackey_glass(timesteps, tau=17, a=0.2, b=0.1, n=10, x0=1.2, h=1, seed=None)

# Define the time step of your Mackey-Glass system
dt = 0.00001

# Compute the equivalent sampling rate
sampling_rate = 1 / dt

%matplotlib inline
plt.figure(figsize=(16,5))
plt.plot(range(1000), mg_inputs[:1000])
plt.show()

## Free Spoken Digits Dataset

In [ ]:
from datasets.load_datasets import load_FSDD_dataset

data_dir = 'datasets/fsdd/free-spoken-digit-dataset-master/recordings'  # Path to the extracted dataset
sampling_rate, X_train, X_test, Y_train, Y_test = load_FSDD_dataset(data_dir, seed=SEED)
# Check the shapes of the datasets
print("X_train shape:", X_train.shape)
print("Y_train shape:", Y_train.shape)
print("X_test shape:", X_test.shape)
print("Y_test shape:", Y_test.shape)

# New inputs creation with band filters

Spectrograms_vs_Cochleagrams : 
* https://www.researchgate.net/publication/340510607_Speech_recognition_using_very_deep_neural_networks_Spectrograms_vs_Cochleagrams

## Pretrain dataset

In [ ]:
#take a long time (15min with half the samples, instant with 20 which is enought for pretraining)
X_pretrain = np.concatenate(X_train[:20], axis=0)
print(X_pretrain.shape)

## Spectral density and peak selection

In [ ]:
from datasets.multivariate_generation import generate_multivariate_dataset, extract_peak_frequencies

filtered_peak_freqs = extract_peak_frequencies(X_pretrain.flatten(), sampling_rate, nperseg=1024, visualize=False)

## Applying normal band pass filter on data

In [ ]:
X_pretrain_band, X_train_band, X_test_band = generate_multivariate_dataset(filtered_peak_freqs, X_pretrain, X_train, X_test, sampling_rate, nb_jobs=-1, visualize=True)

## Standardizing the amplitudes

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(-1, 1))

# pretrain
filtered_data = scaler.fit_transform(X_pretrain_band)

In [ ]:
# train
X_train_band = [np.array([scaler.fit_transform(time_serie.reshape(-1, 1)).flatten() for time_serie in x]) for x in X_train_band]

In [ ]:
# test
X_test_band = [np.array([scaler.fit_transform(time_serie.reshape(-1, 1)).flatten() for time_serie in x]) for x in X_test_band]

In [ ]:
X_train_band[0].shape

# Generating reservoirs

## Creating from HADSP + bandfilter

### Plot  pretrain dataset

In [ ]:
# Min window size to get all the dynamics ? 
min_window_size = sampling_rate/filtered_peak_freqs[-1]

min_window_size

In [ ]:
#Compute the moving average 
window_size = 5
weights = np.repeat(1.0, window_size)/window_size
ma = np.array([np.convolve(d, weights, 'valid') for d in (filtered_data)])

#CPlot the two for different frequencies
NB_1 = 3
fig, ax = plt.subplots(2, 1, figsize=(24,12))
ax[0].plot(range(500), filtered_data[NB_1, 1000:1500], label='Time serie')
ax[0].plot(range(500), ma[NB_1, 1000:1500], label='Moving average')
NB_2 = 10
ax[0].legend(fontsize=26)
ax[1].plot(range(500), filtered_data[NB_2, 1000:1500], label='Time serie')
ax[1].plot(range(500), ma[NB_2, 1000:1500], label='Moving average')

ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[0].tick_params(axis='both', labelsize=26)
ax[1].tick_params(axis='both', labelsize=26)


# draw vertical lines to represent the window for some points
for x in range(100, 500, 100):
    ax[0].axvspan(x, x+window_size, color='g', alpha=0.2)
for x in range(100, 500, 100):
    ax[1].axvspan(x, x+window_size, color='g', alpha=0.2)

plt.show()

## Construct matrices

### Shared parameters

In [ ]:
import math 

# We want the size of the reservoir to be at least 200
K = math.ceil(200 / filtered_peak_freqs.shape[0])
n = filtered_peak_freqs.shape[0] * K
n

In [ ]:
import reservoir.reservoir
from reservoir.reservoir import update_reservoir
from reservoir.reservoir import init_matrices
from connexion_generation.utility import compute_synaptic_change
from connexion_generation.bounded_adsp import bounded_adsp


class TwoDimArrayWrapper:
    def __init__(self, input_data):
        if input_data.ndim != 2:
            raise ValueError("Expected a 2D array.")
        self.input_data = input_data
        self.shape = input_data.shape
        self.size = input_data.shape[1]
        self.flat_data = input_data.flatten()

    def __getitem__(self, key):
        # Handle single element access
        return self.input_data[:, key]
        
def run_HADSP_algorithm(W, Win, bias, input_data, visualize=False):
    # last_state
    state = np.random.uniform(0, 1, n)
    state_history = []
    
    total_add = 0
    total_prun = 0
    add = []
    prun = []
    step=0

    for i in range(INCREMENT*5):
        state = update_reservoir(W, Win, input_data[i], state, leaky_rate, bias, activation_function)
        state_history.append(state)

    # size of simulation 
    number_steps = int((input_data.size-INCREMENT*5)/INCREMENT)
    for k in range(number_steps): 
        delta_z = compute_synaptic_change(state_history[-INCREMENT:], target_rate, growth_parameter, average="WHOLE")
        W, _, nb_new_add, nb_new_prun = bounded_adsp(W, state, delta_z, VALUE)
    
        for i in range(INCREMENT):
            state = update_reservoir(W, Win, input_data[INCREMENT*(k+5)+i], state, leaky_rate, bias, activation_function)
            state_history.append(state)
            
        total_add += nb_new_add
        total_prun += nb_new_prun
        add.append(total_add)
        prun.append(total_prun)
        step += 1
        
    add = np.array(add)
    prun = np.array(prun)

    if visualize:
        plt.figure()
        plt.plot(np.arange(step)*INCREMENT, add, label="total number of added connexion")
        plt.plot(np.arange(step)*INCREMENT, prun, label="total number of prunned connexion")
        plt.plot(np.arange(step)*INCREMENT, add-prun, label="difference")
        plt.plot(np.arange(step)*INCREMENT, [0]*step, linestyle=(0, (1, 10)))
        plt.legend()
        plt.grid()
    return W

def initialise_and_train(input_scaling, n, bias_scaling, seed, training_set, visualize=False):
    Win, W, bias = init_matrices(n, 1, 0, seed=seed)
    bias *= bias_scaling
    Win *= input_scaling
    
    W = run_HADSP_algorithm(W, Win, bias, training_set, visualize=visualize)

    connectivity =  W.count_nonzero() / (W.shape[0] * W.shape[1])
    eigen = sparse.linalg.eigs(W, k=1, which="LM", maxiter=W.shape[0] * 20, tol=0.1, return_eigenvectors=False)
    sr = max(abs(eigen))
    
    return Win, W, bias, connectivity, sr



In [ ]:
from reservoirpy.nodes import Reservoir, Ridge, Input, ESN
from scipy.sparse import csr_matrix
from joblib import Parallel, delayed
from sklearn.metrics import mutual_info_score
from sklearn.metrics import accuracy_score

import tqdm
tqdm.tqdm = lambda *args, **kwargs: args[0]  # Mocks tqdm

N_JOBS = 4

def train_and_predict_model(W, Win, bias, activation_function, ridge_coef, compute_data, predict_data):
    # To remember : 
    #  For reservoirpy   pre_s = W @ r + Win @ (u + noise_gen(dist=dist, shape=u.shape, gain=g_in)) + bias
    
    reservoir = Reservoir(units=n, 
                          W =csr_matrix(W), 
                          Win=csr_matrix(np.diag(Win.toarray().flatten())), 
                          bias=csr_matrix(bias).T, 
                          activation=activation_function,
                          equation='external'
                         )
    readout = Ridge(ridge=ridge_coef)
    model = ESN(reservoir=reservoir, readout=readout)
    
    states_train = []

    def compute_state(x):
        return reservoir.run(x, reset=True)[-1, np.newaxis].flatten()
    
    states_train = Parallel(n_jobs=N_JOBS)(delayed(compute_state)(x) for x in compute_data)

    readout.fit(np.array(states_train), Y_train)

    Y_pred = []
    def predict(x):
        states = reservoir.run(x, reset=True)
        y = readout.run(states[-1, np.newaxis])
        return y

    Y_pred = Parallel(n_jobs=N_JOBS)(delayed(predict)(x) for x in predict_data)

    return Y_pred

def compute_score(Y_pred, Y_test, model_name):
    Y_pred_class = [np.argmax(y_p) for y_p in Y_pred]
    Y_test_class = [np.argmax(y_t) for y_t in Y_test]

    score = accuracy_score(Y_test_class, Y_pred_class)

    print(f"Accuracy for {model_name}: {score * 100:.3f} %")
    return score

### Multivariate matrices

In [ ]:
frequency_bands = np.repeat(filtered_data, K, axis=0)

frequency_bands = TwoDimArrayWrapper(frequency_bands)

In [ ]:
# We want the size of the reservoir to be at least 200
caca = []
caca_test = []
for i in range(len(X_train_band)):
    #caca.append(np.repeat(X_train[i], K, axis=1))
    caca.append(np.repeat(X_train_band[i], K, axis=0).T) # axis still depend of X_train shape
for i in range(len(X_test_band)):
    #caca_test.append(np.repeat(X_test[i], K, axis=1))
    caca_test.append(np.repeat(X_test_band[i], K, axis=0).T)


# Create a list to store the arrays

pipi = []
pipi_test = []

for i in range(len(X_train)):
    pipi.append(np.repeat(X_train[i], n, axis=1))
    #pipi.append(np.repeat(X_train[i], n, axis=0).T) # axis still depend of X_train shape
    
for i in range(len(X_test)):
    pipi_test.append(np.repeat(X_test[i], n, axis=1))
    #pipi_test.append(np.repeat(X_test[i], n, axis=0).T)
    
pipi[0].shape

In [ ]:
scores_hadsp_multi = []
scores_random_multi = []
scores_hadsp_uni = []
scores_random_uni = []

INCREMENT = 5
VALUE = 0.05
target_rate = 0.7
growth_parameter = 0.3

bias_scaling = 1
leaky_rate = 1
ridge_coef = 1e-6

for input_scaling in [0.1, 0.2, 0.3]: 
    # HADSP + multi
    Win_hasp_band, W_hasp_band, bias_hasp_band, connectivity_band, sr_adsp = initialise_and_train(input_scaling, n, bias_scaling, SEED, frequency_bands)
    
    # 3rd (random + multi)
    Win_3, W_3, bias_3 =  init_matrices(n, 1, connectivity_band, sr_adsp)
    bias_3= bias_3*bias_scaling
    Win_3= Win_3*input_scaling
    
    eigen_3 = sparse.linalg.eigs(W_3, k=1, which="LM", maxiter=W_3.shape[0] * 20, tol=0.1, return_eigenvectors=False)
    sr_3 = max(abs(eigen_3))
    
    # HADSP + uni
    Win_hadsp_uni, W_hadsp_uni, bias_hadsp_uni, connectivity_hadsp_uni, sr_hadsp_uni = initialise_and_train(input_scaling, n, bias_scaling, SEED, X_pretrain.flatten())

    # random + uni
    Win_normal, W_normal, bias_normal =  init_matrices(n, 1, connectivity_hadsp_uni)
    bias_normal= bias_normal*bias_scaling
    Win_normal= Win_normal*input_scaling   

    eigen_normal = sparse.linalg.eigs(W_normal, k=1, which="LM", maxiter=W_normal.shape[0] * 20, tol=0.1, return_eigenvectors=False)
    sr_normal = max(abs(eigen_normal))


    
    # Spectral radius normalisation
    normal_sr = 1.2
    W_normal = W_normal/sr_normal*normal_sr
    W_hadsp_uni = W_hadsp_uni/sr_hadsp_uni*normal_sr
    W_3 = W_3/sr_3*normal_sr
    W_hasp_band = W_hasp_band/sr_adsp*normal_sr

    
    #  HADSP + multivariate dataset
    Y_pred_hasp_band = train_and_predict_model(W_hasp_band, Win_hasp_band, bias_hasp_band, activation_function, ridge_coef, caca, caca_test)
    # random + multivariate dataset
    Y_pred_3 = train_and_predict_model(W_3, Win_3, bias_3, activation_function, ridge_coef, caca, caca_test)
    
    # HADSP + univariate dataset
    Y_pred_hadsp_uni = train_and_predict_model(W_hadsp_uni, Win_hadsp_uni, bias_hadsp_uni, activation_function, ridge_coef, pipi, pipi_test)
    # random + multivariate dataset
    Y_pred_normal = train_and_predict_model(W_normal, Win_normal, bias_normal, activation_function, ridge_coef, pipi, pipi_test)

    
    scores_hadsp_multi.append(compute_score(Y_pred_hasp_band, Y_test, "HADSP multi"))
    scores_random_multi.append(compute_score(Y_pred_3, Y_test, "random multi"))

    scores_hadsp_uni.append(compute_score(Y_pred_hadsp_uni, Y_test, "HADSP uni"))
    scores_random_uni.append(compute_score(Y_pred_normal, Y_test, "random uni"))


In [ ]:

# Compute the mean and standard deviation of the scores
mean_hadsp_multi = np.mean(scores_hadsp_multi)
std_hadsp_multi = np.std(scores_hadsp_multi)
print(f'Mean and Std for HADSP multi: {mean_hadsp_multi}, {std_hadsp_multi}')

mean_random_multi = np.mean(scores_random_multi)
std_random_multi = np.std(scores_random_multi)
print(f'Mean and Std for random multi: {mean_random_multi}, {std_random_multi}')

mean_hadsp_uni = np.mean(scores_hadsp_uni)
std_hadsp_uni = np.std(scores_hadsp_uni)
print(f'Mean and Std for HADSP uni: {mean_hadsp_uni}, {std_hadsp_uni}')

mean_random_uni = np.mean(scores_random_uni)
std_random_uni = np.std(scores_random_uni)
print(f'Mean and Std for random uni: {mean_random_uni}, {std_random_uni}')